# Programming language interoperability (Interop)

## Python

In [2]:
using PythonCall

In [3]:
@pyeval "3+3"

Python int: 6

In [4]:
np = pyimport("numpy")

Python module: <module 'numpy' from '/Users/crstnbr/repos/JuliaWorkshops/JuliaHLRS22/.CondaPkg/env/lib/python3.10/site-packages/numpy/__init__.py'>

In [5]:
np.linalg.eigvals(np.random.rand(5,5))

Python ndarray:
array([ 2.86925947+0.j        ,  0.26579958+0.27357631j,
        0.26579958-0.27357631j, -0.11501699+0.j        ,
       -0.47982533+0.j        ])

In [6]:
M = rand(5,5)
np.linalg.eigvals(M)

Python ndarray: array([ 2.2818201 , -0.44745434,  0.0324065 ,  0.54809614,  0.30481062])

In [7]:
@pyexec """
global sinpi, np
import numpy as np

def sinpi(x):
    return np.sin(np.pi * x)
"""

In [8]:
py_sinpi(x) = pyconvert(Float64, @pyeval("sinpi")(x))

py_sinpi (generic function with 1 method)

In [9]:
py_sinpi(10)

-1.2246467991473533e-15

In [10]:
using BenchmarkTools
@btime py_sinpi(10);
@btime sinpi(10); # built-in Julia function

  1.705 μs (3 allocations: 48 bytes)
  1.984 ns (0 allocations: 0 bytes)


## C

In [11]:
c_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
""";

Compile to a shared library by piping `c_code` to gcc:

In [12]:
using Libdl
const Clib = tempname() * "." * Libdl.dlext

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $Clib -`, "w") do f
    print(f, c_code)
end

In [13]:
Clib

"/var/folders/1d/sbf5s24x6y306tgv_4hh06y00000gn/T/jl_JCFwT8lwJY.dylib"

Binding the function from the shared library:

In [14]:
c_sum(X::Array{Float64}) = @ccall Clib.c_sum(length(X)::Csize_t, X::Ptr{Float64})::Float64

c_sum (generic function with 1 method)

In [15]:
c_sum(rand(10))

5.470145790255269

In [16]:
x = rand(10)
@btime c_sum($x);

  8.307 ns (0 allocations: 0 bytes)


## Mixing Julia, Python, and C

Julia (`real`), Python/numpy (`py_sinpi`), C (`c_sum`)

In [17]:
x = rand(10);

In [18]:
abs(py_sinpi(c_sum(x)))

0.5172922600550105

In [19]:
@btime abs(py_sinpi(c_sum($x)));

  1.673 μs (3 allocations: 48 bytes)


See [JuliaInterop](https://github.com/JuliaInterop) for more, such as [RCall.jl](https://github.com/JuliaInterop/RCall.jl), [JavaCall.jl](https://github.com/JuliaInterop/JavaCall.jl), and [MATLAB.jl](https://github.com/JuliaInterop/MATLAB.jl).

# Julia Microbenchmark: Summation

Let's look at and benchmark the sum function:

$$\mathrm{sum}(x) = \sum_{i=1}^n x_i$$

In [1]:
x = rand(10^7);

In [2]:
sum(x)

5.0009601759730885e6

In [3]:
d = Dict() # to store the measurement results

Dict{Any, Any}()

## Python

In [4]:
using BenchmarkTools
using PythonCall

### numpy

In [5]:
np = pyimport("numpy")

Python module: <module 'numpy' from '/Users/crstnbr/repos/JuliaWorkshops/JuliaHLRS22/.CondaPkg/env/lib/python3.10/site-packages/numpy/__init__.py'>

In [6]:
numpy_sum = np.sum

Python function: <function sum at 0x111d95ab0>

In [7]:
b = @benchmark $numpy_sum($x)

BenchmarkTools.Trial: 733 samples with 1 evaluation.
 Range (min … max):  6.051 ms … 12.138 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     6.224 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.808 ms ±  1.132 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▇▅▄▂▁         ▁    ▁                                       
  ████████▇▇▇▇▇▇▆██▇▆▆█▇▁▅██▇▇█▇▆▄█▇▇▁▄▇▅▅▇▄▄▁▁▇▄▅▆▆▆▁▄▆▁▄▁▄ ▇
  6.05 ms      Histogram: log(frequency) by time     10.6 ms <

 Memory estimate: 928 bytes, allocs estimate: 23.

In [8]:
d["Python (numpy)"] = minimum(b.times) / 1e6

6.051352

### hand-written

In [9]:
@pyexec """
global mysum

def mysum(a):
    s = 0.0
    for x in a:
        s = s + x
    return s
"""

In [10]:
mysum_py = @pyeval("mysum")

Python function: <function mysum at 0x115f61630>

In [11]:
x_py = pylist(x);

In [12]:
b = @benchmark $mysum_py($x_py)

BenchmarkTools.Trial: 15 samples with 1 evaluation.
 Range (min … max):  342.592 ms … 346.976 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     344.543 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   344.512 ms ±   1.161 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▁▁          ▁     ▁  ▁▁   ▁▁▁ ▁ █▁                   ▁      ▁  
  ██▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁█▁▁██▁▁▁███▁█▁██▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁█ ▁
  343 ms           Histogram: frequency by time          347 ms <

 Memory estimate: 16 bytes, allocs estimate: 1.

In [13]:
d["Python (hand-written)"] = minimum(b.times) / 1e6

342.591904

### built-in

In [14]:
# get the Python built-in "sum" function:
pysum = pybuiltins.sum

Python builtin_function_or_method: <built-in function sum>

In [15]:
b = @benchmark $pysum($x_py)

BenchmarkTools.Trial: 96 samples with 1 evaluation.
 Range (min … max):  48.584 ms … 86.104 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     50.789 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   52.191 ms ±  5.779 ms  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▁▃▂▆▃▃▂      ▁                                              
  ████████▇▁▅▅▅▅█▁▅▁▁▅▁▁▅▁▁▁▁▁▁▁▅▁▁▅▁▁▁▁▁▁▁▁▁▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅ ▁
  48.6 ms      Histogram: log(frequency) by time      78.8 ms <

 Memory estimate: 16 bytes, allocs estimate: 1.

In [16]:
d["Python (built-in)"] = minimum(b.times) / 1e6

48.584259

## C

### hand-written

In [17]:
c_code = """
#include <stddef.h>
double c_sum(size_t n, double *X) {
    double s = 0.0;
    for (size_t i = 0; i < n; ++i) {
        s += X[i];
    }
    return s;
}
""";

In [18]:
# compile to a shared library by piping C_code to gcc:
# (only works if you have gcc installed)
using Libdl
const Clib = tempname() * "." * Libdl.dlext

"/var/folders/1d/sbf5s24x6y306tgv_4hh06y00000gn/T/jl_VUJIPOdsBE.dylib"

In [19]:
open(`gcc -fPIC -O3 -msse3 -xc -shared -o $Clib -`, "w") do f
    print(f, c_code)
end

In [20]:
c_sum(X::Array{Float64}) = @ccall Clib.c_sum(length(X)::Csize_t, X::Ptr{Float64})::Float64

c_sum (generic function with 1 method)

In [21]:
c_sum(x) ≈ sum(x)

true

In [22]:
b = @benchmark c_sum($x)

BenchmarkTools.Trial: 317 samples with 1 evaluation.
 Range (min … max):  15.570 ms …  19.731 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     15.629 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   15.799 ms ± 522.341 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  █▅▃▁▁▁                                                        
  ██████▆▅▆▇▆▇▆▄▆▄▄▆▁▄▁▁▄▅▄▄▁▄▁▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▁▁▁▁▁▁▁▄▄ ▆
  15.6 ms       Histogram: log(frequency) by time      18.8 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [23]:
d["C"] = minimum(b.times) / 1e6

15.56973

### hand-written (with `-fast-math`)

In [24]:
const Clib_fastmath = tempname() * "." * Libdl.dlext

# The same as above but with a -ffast-math flag added
open(`gcc -fPIC -O3 -msse3 -xc -shared -ffast-math -o $Clib_fastmath -`, "w") do f
    print(f, c_code) 
end

# define a Julia function that calls the C function:
c_sum_fastmath(X::Array{Float64}) = @ccall Clib_fastmath.c_sum(length(X)::Csize_t, X::Ptr{Float64})::Float64

c_sum_fastmath (generic function with 1 method)

In [25]:
b = @benchmark c_sum_fastmath($x)

BenchmarkTools.Trial: 965 samples with 1 evaluation.
 Range (min … max):  4.716 ms …  13.175 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     4.953 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.172 ms ± 630.024 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▇▅█▆▂                                                       
  ▅██████▆▅▄▃▄▄▃▃▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▃▂▂▂▃▂▂▂▂▂▂▂▁▁▁▂▂▁▁▂▂▁▂▁▂ ▃
  4.72 ms         Histogram: frequency by time        7.74 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [26]:
d["C (fastmath)"] = minimum(b.times) / 1e6

4.716007

## Julia

### built-in

In [27]:
b = @benchmark sum($x)

BenchmarkTools.Trial: 950 samples with 1 evaluation.
 Range (min … max):  4.655 ms …   9.036 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     5.144 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   5.253 ms ± 464.013 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

       ▂▄▂▃█▆▅▄▃▁▁                                             
  ▃▃▄▆▃███████████▇██▄▄▃▃▃▃▃▃▃▁▂▂▃▃▂▃▂▂▂▃▂▃▁▃▁▂▂▂▁▂▃▂▁▁▂▂▂▂▂▂ ▃
  4.66 ms         Histogram: frequency by time        7.12 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [28]:
d["Julia (built-in)"] = minimum(b.times) / 1e6

4.655103

### built-in (with `Vector{Any}`)

In [29]:
x_any = Vector{Any}(x)
b = @benchmark sum($x_any)

BenchmarkTools.Trial: 16 samples with 1 evaluation.
 Range (min … max):  288.181 ms … 345.246 ms  ┊ GC (min … max): 0.00% … 12.99%
 Time  (median):     321.014 ms               ┊ GC (median):    9.26%
 Time  (mean ± σ):   323.187 ms ±  13.981 ms  ┊ GC (mean ± σ):  8.48% ±  3.45%

  ▁                         ▁ ▁ ▁▁▁█  █▁      ▁   ▁       ▁   █  
  █▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁█▁████▁▁██▁▁▁▁▁▁█▁▁▁█▁▁▁▁▁▁▁█▁▁▁█ ▁
  288 ms           Histogram: frequency by time          345 ms <

 Memory estimate: 152.59 MiB, allocs estimate: 9999999.

In [30]:
d["Julia (built-in, Any)"] = minimum(b.times) / 1e6

288.181248

### hand-written

In [31]:
function mysum(A)
    s = zero(eltype(A)) # the correct type of zero for A
    for a in A
        s += a
    end
    return s
end

mysum (generic function with 1 method)

In [32]:
b = @benchmark mysum($x)

BenchmarkTools.Trial: 307 samples with 1 evaluation.
 Range (min … max):  15.837 ms …  20.089 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     16.097 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   16.281 ms ± 580.425 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▃▇▇█▆▅▄▂                                                      
  ████████▆█▅█▃▅▅▃▃▃▁▄▃▃▄▃▃▂▁▂▃▁▂▂▁▂▁▁▁▂▁▂▂▂▁▁▁▁▁▁▂▁▃▁▁▁▁▁▁▂▁▃ ▃
  15.8 ms         Histogram: frequency by time         18.8 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [33]:
d["Julia (hand-written)"] = minimum(b.times) / 1e6

15.836575

### hand-written (with `@fastmath`)

In [34]:
function mysum_fastmath(A)
    s = zero(eltype(A)) # the correct type of zero for A
    @fastmath for a in A
        s += a
    end
    return s
end

mysum_fastmath (generic function with 1 method)

In [35]:
b = @benchmark mysum_fastmath($x)

BenchmarkTools.Trial: 1055 samples with 1 evaluation.
 Range (min … max):  4.402 ms …   8.725 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     4.633 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.730 ms ± 366.623 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

   ▄▇▅▄▃▄█▃▂▂▁                                                 
  ▅████████████▅▆▄▃▄▃▃▃▃▃▃▃▃▂▃▃▃▃▁▂▂▂▂▂▃▂▂▃▃▂▂▂▂▁▁▃▁▂▂▂▁▂▂▂▁▂ ▃
  4.4 ms          Histogram: frequency by time         6.2 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

In [36]:
d["Julia (hand-written, fastmath)"] = minimum(b.times) / 1e6

4.401845

## Summary

In [38]:
for (key, value) in sort(collect(d), by=x->x[2])
    println(rpad(key, 30, "."), lpad(round(value, digits=2), 10, "."))
end

Julia (hand-written, fastmath).......4.4
Julia (built-in)....................4.66
C (fastmath)........................4.72
Python (numpy)......................6.05
C..................................15.57
Julia (hand-written)...............15.84
Python (built-in)..................48.58
Julia (built-in, Any).............288.18
Python (hand-written).............342.59


And of course, our hand-written Julia implementation is type-generic!

In [39]:
mysum_fastmath(rand(ComplexF64, 10))

5.611325833972627 + 4.843774096110263im

# Supplement: What about other functions?

### Log

In [40]:
@which log(1.2)

log(x::Float64) in Base.Math at special/log.jl:267

In [41]:
using BenchmarkTools

# uses the system C library
clog(x) = ccall(:log, Float64, (Float64,), x)
# uses LLVM's log
llvmlog(x) =  ccall(Symbol("llvm.log.f64"), llvmcall, Float64, (Float64,), x)

@btime log(1.2)    
@btime clog(1.2)    
@btime llvmlog(1.2);

  1.983 ns (0 allocations: 0 bytes)
  8.705 ns (0 allocations: 0 bytes)
  3.533 ns (0 allocations: 0 bytes)


### Exp

In [42]:
@which exp(1.2)

exp(x::Union{Float16, Float32, Float64}) in Base.Math at special/exp.jl:327

In [43]:
using BenchmarkTools

# uses the system C library
cexp(x) = ccall(:exp, Float64, (Float64,), x)
# uses LLVM's
llvmexp(x) =  ccall(Symbol("llvm.exp.f64"), llvmcall, Float64, (Float64,), x)

@btime exp(1.2);
@btime cexp(1.2);
@btime llvmexp(1.2);

  1.984 ns (0 allocations: 0 bytes)
  9.916 ns (0 allocations: 0 bytes)
  3.525 ns (0 allocations: 0 bytes)


### Matrix multiplication

In [1]:
N = 10
C = zeros(N,N);
A = rand(N,N);
B = rand(N,N);

In [2]:
using LinearAlgebra

mul!(C, A, B); # "built-in", calls underlying BLAS/LAPACK

In [3]:
C ≈ A * B

true

In [4]:
using BenchmarkTools

function mul_naive!(C, A, B)
    for m in axes(A,1)
        for n in axes(B,2)
            Cmn = zero(eltype(C))
            for k in axes(A,2)
               @inbounds Cmn += A[m,k] * B[k,n]
            end
            C[m,n] = Cmn
        end
   end
end

mul_naive! (generic function with 1 method)

In [5]:
mul_naive!(C,A,B)
C ≈ A * B

true

[LoopVectorization.jl](https://github.com/chriselrod/LoopVectorization.jl)

In [6]:
using LoopVectorization

function mul_turbo!(C, A, B)
    @turbo for m in axes(A,1)
        for n in axes(B,2)
            Cmn = zero(eltype(C))
            for k in axes(A,2)
               @inbounds Cmn += A[m,k] * B[k,n]
            end
            C[m,n] = Cmn
        end
   end
end

mul_turbo! (generic function with 1 method)

In [7]:
mul_turbo!(C,A,B)
C ≈ A * B

true

In [8]:
c_code = """
#include <stddef.h>
#include <math.h>

void gemm(double* restrict C, double* restrict A, double* restrict B, long M, long K, long N){
  for (long i = 0; i < M*N; i++){
    C[i] = 0.0;
  }
  for (long n = 0; n < N; n++){
    for (long k = 0; k < K; k++){
      for (long m = 0; m < M; m++){
	C[m + n*M] += A[m + k*M] * B[k + n*K];
      }
    }
  }
  return;
}
""";

In [9]:
# compile to a shared library by piping C_code to gcc:
# (only works if you have gcc installed)
using Libdl
const Clib_gemm = tempname() * "." * Libdl.dlext

open(`gcc -fPIC -O3 -msse3 -xc -shared -o $Clib_gemm -`, "w") do f
    print(f, c_code)
end

c_gemm(C::Array{Float64},A::Array{Float64},B::Array{Float64}) = @ccall Clib_gemm.gemm(C::Ptr{Float64}, A::Ptr{Float64}, B::Ptr{Float64}, size(A,1)::Clong, size(A,2)::Clong, size(B,2)::Clong)::Cvoid

c_gemm (generic function with 1 method)

In [10]:
c_gemm(C, A, B)
C ≈ A*B

true

In [12]:
@btime mul_naive!($C,$A,$B);
@btime mul_turbo!($C,$A,$B);
@btime mul!($C, $A, $B);
@btime c_gemm($C, $A, $B)

  932.600 ns (0 allocations: 0 bytes)
  134.767 ns (0 allocations: 0 bytes)
  399.935 ns (0 allocations: 0 bytes)
  565.205 ns (0 allocations: 0 bytes)


**Note for larger `N`:** BLAS is multithreaded for larger `N`. In this case our `mul_avx!` can be slower than `mul!`.